In [1]:
# Importação de bibliotecas
import pandas as pd

import googledriver as gdriver

In [2]:
# Objetos
fonte = 'anda'
dataset_name = 'fertilizantes_sinteticos'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [56]:
id_arquivo = '1dwglTy1GCPjTEZW1z7-Hb5_DQIwGmdHzOVBMw0FEIsg' # ID adquirida na URL
url_gdrive = f'https://docs.google.com/spreadsheets/d/{id_arquivo}/export?format=xlsx&t?gid=0#gid=0' # Endereço específico para exportação de dados planilhares

saida = f'{raiz}/bronze_{fonte}_{dataset_name}.xlsx'

gdriver.download(url_gdrive, saida, None)

'/home/eduardo/repositorios/IS_Agro/databases/bronze_anda_fertilizantes_sinteticos.xlsx'

### Prata

NPK na forma de fertilizante sintético incorporado ao solo (t)

In [5]:
# Criação de dataframe vazio para uso posterior
df_prata = pd.DataFrame()

# Abas das tabelas na planilha bronze
tabelas = pd.ExcelFile(f'{raiz}/bronze_{fonte}_{dataset_name}.xlsx').sheet_names

In [6]:
for fertilizante in tabelas:
        df = (pd.read_excel(f'{raiz}/bronze_{fonte}_{dataset_name}.xlsx',
                            sheet_name=fertilizante,
                            header=2,
                            nrows=28,
                            na_values=pd.NA
                        )
                .query('uf != "Brasil"')
                .drop(columns='uf')
                .dropna(axis='columns', thresh=10)
                .melt(id_vars='geocodigo',
                      var_name='data',
                      value_name='valor'
                      )
                .dropna(subset='valor')
                .query('valor != 0') # Exclusão de registros que possuam valor zerado
                .assign(**{
                        'data': lambda df: pd.to_datetime(df['data'], format='%Y'), # Conversão de número inteiro para formato de data
                        'elemento': f'{fertilizante}', # Criação de nova coluna de informação do que representa o dado
                        'fonte': 'Fertilizantes Sintéticos'
                        })
                )
        df_prata = (pd.concat([df_prata, df],
                         ignore_index=True)
                )

df_prata = (df_prata.convert_dtypes()
            .sort_values(by=['geocodigo', 'data']) # Organização dos dados
            .set_index(['geocodigo', 'data', 'fonte', 'elemento'])
            )

df_prata.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}.parquet')

### Ouro

Elementos químicos na forma de fertilizante entregue ao mercado (t)

Parâmetros

Teor (%)

Cálculos:
Valor Estadual N * (Parâmetro N/100) * ((100 - fertilizantes_sinteticos)/100)

Valor Estadual P2o5 * (Parâmetro P)/100

Valor Estadual K2o * (Parâmetro K)/100


Brasil = mil t

In [7]:
parametro_N = 100
parametro_P = 43.7
parametro_K = 83

# Perdas por volatilização do N
perdas_fertiliz_sintetico = 0

In [8]:
df = (pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}.parquet')
      .reset_index()
      .assign(**{
          'valor': lambda df: df.apply(lambda linha: linha.valor*(parametro_N/100)*((100 - perdas_fertiliz_sintetico)/100) if linha.elemento == 'Fert_Sint_N'
                                     else linha.valor*(parametro_P/100) if linha.elemento == 'Fert_Sint_P'
                                     else linha.valor*(parametro_K/100),
                                       axis = 1)
                                       })
      .replace({
          'Fert_Sint_N': 'N',
          'Fert_Sint_P': 'P',
          'Fert_Sint_K': 'K'
          })
      .convert_dtypes()
    )

In [15]:
df = (
    df
    .rename(columns={
        'geocodigo':'state',
        'data': 'date',
        'fonte': 'label',
        'elemento':'nutrient',
        'valor':'value'
    })
)

df.to_csv(f'{raiz}/ouro_npk_fert_sintetico.csv')